![INSA](https://gi.insa-lyon.fr/sites/all/themes/insa_satellites/logo.png)

# GI-5-DSC - Data Science: Introduction to data exploration & visualization
***


The purpose of this tutorial is to familiarize ourselves with data exploration and visualization. 
We will start using Python libraries to manipulate data.
For that, we will work on a case study which consists in the analysis of the data of the Lyon Metropole bike sharing platform (Vélo'v). 

The data we are going to use come from the [Grand Lyon open data platform](https://data.grandlyon.com). They are made available free of charge by Lyon Metropole and can be downloaded in different formats: https://data.grandlyon.com/jeux-de-donnees/historique-disponibilites-stations-velo-v-metropole-lyon/donnees.

From the Grand Lyon website, only the last 7 days are available.


![site du grand lyon](https://perso.liris.cnrs.fr/lmoncla/GI-5-DSC/fig/grandlyon.png)


In this tutorial, you will have at your disposal the entire dataset for the year 2021.


The objectives of this tutorial are: 

* Load the dataset into a dataframe
* Explore and visualize the data
* Analyze the data: query the dataset to generate graphs and maps


***



## 1. Set up the environment: import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone

import folium
import plotly
import plotly.express as px
import geopandas

## 2. Getting the data

To overcome the limit of 7 days of availability on the Grand Lyon website, I have developed a script that automatically retrieves and stores the data every day. You will thus have access to the data for the whole year 2021. 
I also propose the data in a CSV format (easier to load into a dataframe than the original JSON format). 


If you cloned the [github repository](https://github.com/ludovicmoncla/insa-5gi-dsc-tutorials) you already have the data and you can go directly to section 2.2.


### 2.1 Download the data

All the data used in this tutorial are available on the `data` folder of the [github repository](https://github.com/ludovicmoncla/insa-5gi-dsc-tutorials/tree/main/data).

* Download the two zip files containing the data:
1. data-stations.zip
2. data-bikes.zip

These 2 zip files contain a CSV file containing respectively the list of velov stations (and their location) and the list of availabilities of each station by 30 minutes.



### 2.2. Load the data


In this tutorial we will not use a DBMS. The objective is to load the data in memory into a Python structure and to query it directly. 

There are two types of data:
1. vélo'v stations (station id, latitude, longitude),
2. their history (station id, year, month, day, hour, minute, date, bikes available, available slots).

To manipulate these data we will use the  [dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) of the Pandas library.

Pandas is a Python library specialized in data analysis and manipulation. It provides in particular an object of type 'dataframe' which makes it possible to carry out operations of preprocessing and filtering which we will use to request the data.

The first objectives are the following:

1. To store in a first dataframe the list of velo'v stations and their associated latitude / longitude coordinates.
2. To store in a second dataframe for each station and each timestamp the following data: 
    * id of the station
    * year
    * month
    * day
    * hour
    * minute
    * complete date (original format)
    * number of available bikes
    * number of free slots
    * number of departures in the last 30 minutes
    * number of arrivals in the last 30 minutes

To load the data you just have to use the method [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) of the `Pandas` library. It takes as a parameter the path of the file you want to load. This file can be of 2 formats, either directly a CSV file, or a ZIP file containing a CSV. In our case, it is useless to unzip the previously downloaded archives.



In [4]:
## Load the data from the stations into a dataframe
df_stations = pd.read_csv('data/data-stations.zip')

## Create the dataframe with the history data
df_bikes = pd.read_csv('data/data-bikes.zip')

In [ ]:
## Check the type of our variable
type(df_stations)

In [ ]:
## Display the list of columns
df_stations.columns

In [ ]:
## Print the five first rows
df_stations.head()

* How many velo'v stations are there ?

In [ ]:
## Show the size of the dataframe
## The shape method returns the dimensions (rows / columns)
print(df_stations.shape)

## The len() function returns the number of rows
print(len(df_stations))

In [ ]:
## Show the first rows
df_bikes.head()

### 2.2. First look at the history data

In [ ]:
## Display some information about the data
df_bikes.info()

In [ ]:
# Reduction of the size in memory
## we transform the type of the columns into integer or float when it is necessary
df_bikes.bikes = df_bikes.bikes.apply(lambda x: int(float(x)))
df_bikes.bike_stands = df_bikes.bike_stands.apply(lambda x: np.int32(float(x)))

df_bikes['year'] = df_bikes['year'].astype('int16')
df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']] = df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']].astype('int8')


In [ ]:
## Display some information about the data
df_bikes.info()

In [ ]:
## Display some stats about the dataframe
df_bikes.describe()

In [ ]:
## Show 5 rows randomly selected
df_bikes.sample(5)

### 2.3. Handling of a dataframe

In [ ]:
## Get data from one column
df_bikes['time']

In [ ]:
## Get data from one column (other method using .)
df_bikes.time

In [ ]:
## Get data from a list of columns
df_bikes[['time', 'bikes']]

In [ ]:
## Get the value from a set of column as a numpy array
df_bikes[['time', 'bikes']].values

A column (or variable) is a data vector (Series in the Pandas library terminology).

In [ ]:
## Display the first values of a specific column
df_bikes['time'].head()

In [ ]:
## Display the last values of a specific column
df_bikes['time'].tail()

In [ ]:
## Sort the values from a specific column
df_bikes['time'].sort_values()

In [ ]:
## We can also sort an entire dataframe
## Sort the dataframe using the station id and the time
df_bikes.sort_values(by=['id_velov', 'time'])

In [ ]:
## Alternative to reset the row indexes to zero
df_bikes = df_bikes.sort_values(by=['id_velov', 'time']).reset_index(drop=True)
df_bikes

In [ ]:
## Display the number of rows for each distinct value in a specific column
df_bikes['id_velov'].value_counts()

In [ ]:
## A column being a vector it is possible to use indices to access the elements
## Display of the first value of the time column
df_bikes['time'][0]

In [ ]:
## Display of the first 3 values of the time column
df_bikes['time'][0:3]

#### 2.3.1 Iterate over columns

Iterations on variables can be done via a loop, or via the use of callback functions called with an `.apply()` function.


In [ ]:
## Loop over all columns to display their name and type
for col in df_bikes.columns:
    print(col, ": ", df_bikes[col].dtype)

#### 2.3.2 Iterate over rows (\*\*not recommended for large dataframes**)


It is possible to browse the rows of a dataframe, but beware, iteration on a dataframe is slow. It is better to use vector operations! If you can't, you should use a callback function called with an `.apply()` function.

Note: you cannot modify a dataframe on which you are looping.



In [ ]:
## For the example, we iterate on the stations dataframe (because the history one is too big)
for index, row in df_stations.iterrows():
    print('ID :', row.id_velov, '\t lat :', row.latitude,'\t lng :', row.longitude)


#### 2.3.3 Indexed access to the data of a DataFrame

The values of the DataFrame can be accessed via indices or ranges of indices. 
The structure then behaves like a matrix. The top left cell has coordinates (0,0).
There are different ways to do this, using `.iloc[,]` is one of the simplest solutions.

Reminder: the `shape()` method is used to get the dimensions (rows and columns) of the DataFrame.


In [ ]:
## Access to the value located in (0,0) (first row, first column)
df_bikes.iloc[0,0]

In [ ]:
## Value located in last row, first column
## Use of negative index
df_bikes.iloc[-1,0]

In [ ]:
## Alternative with shape, value located in last row, first column
## shape[0] returns the number of lines (first dimension)
## you have to reduce by -1 because the first index is equal to 0 otherwise you overflow
df_bikes.iloc[df_bikes.shape[0]-1,0]

In [ ]:
## Display the first 5 values of all columns
## rows => 0:5 (0 à 5 [not included])
## columns = : (all columns)
df_bikes.iloc[0:5,:]

In [ ]:
## With the negative marking, we can easily access the last 5 rows
df_bikes.iloc[-5:,:]

In [ ]:
## 5 first rows and columns 0, 6, 7 and 8
## we have a list of indices for the columns
df_bikes.iloc[0:5,[0,6,7,8]]

#### 2.3.4 Filtering with conditions - Queries

We can isolate subsets of observations that meet criteria defined on columns. We will preferentially use the `.loc[,]` method in this context.

In [ ]:
## List of history data for station 'velov-10001'
df_bikes.loc[df_bikes['id_velov']=="velov-10001",:]

In [ ]:
## For a set of values of the same variable we use the isin() method
df_bikes.loc[df_bikes['id_velov'].isin(['velov-10001','velov-10002']),:]

Logical operators are used to combine conditions.
We use respectively: & for AND, | for OR, and ~ for negation.


In [ ]:
## List of data for station 'velov-10001' and hour = 8
df_bikes.loc[(df_bikes['id_velov']=="velov-10001") & (df_bikes['hour'] == 8),:]

In [ ]:
## List of data from after July
df_bikes.loc[(df_bikes['month'] > 7),:]

In [ ]:
# You can display only part of the columns
# We define the projection in a list
colonnes = ['id_velov','time','bikes','bike_stands']
# that we use as parameter in .loc[]
# example with the same restriction as before
df_bikes.loc[(df_bikes['month'] > 7),colonnes]

#### 2.3.5 Grouping of rows

The use of `groupby()` allows access to the sub-DataFrame associated with each item of the grouping variable. It is then possible to explicitly apply further processing on these subsets of data.

In [ ]:
# grouping data according to the station id
g = df_bikes.groupby('id_velov')

g.size()

In [ ]:
# Calculate the dimension of the sub-DataFrame associated with the station 'velov_10001'
g.get_group('velov-10001').shape

### 2.4. Visualization of station locations

Now that you loaded the data in dataframes and see how to manipulate dataframes, you'll produce your first data viz with these data. As we have geographic data we will display them on a map.


#### 2.4.1 Use of GeoPandas and Plotly libraries

The[GeoPandas](https://geopandas.org/) library is developped to manipulate spatial data. The particularity of GeoPandas is that it allows to manipulate spatial data as if it were traditional data.. 

Compared to a standard `DataFrame`, a `GeoDataFrame` has an additional column: `geometry`. As in a spatial DBMS, this column allows to store the contours (the geometry) of a geographic object. A `GeoDataFrame` object inherits the properties of a `DataFrame` but offers methods adapted to the processing of spatial data.

Thus, in addition to the manipulations already possible with pandas, it will be possible to manipulate the spatial dimension: 
- calculate distances and surfaces,
- quickly aggregate areas (grouping departments into regions for example),
- search for an area from the coordinates of a point,
- convert data into different projection systems,
- display a map.

For the moment we are working on the last item to produce a map of the Velo'v stations.

![stations velov avec GeoPandas](https://perso.liris.cnrs.fr/lmoncla/GI-5-DSC/fig/geopandas_stations.png)

* Display the vélo'v stations on a map. Use the [GeoPandas](https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py) library. You should get the above result.

In [ ]:
## We transform the dataframe of stations into a geodataframe (https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py)
gdf_stations = geopandas.GeoDataFrame(
    df_stations, 
    geometry=geopandas.points_from_xy(df_stations.longitude, df_stations.latitude))


In [ ]:
## Display the first rows of the geodataframe to check the existence of the geometry column
gdf_stations.head()

In [ ]:
## Display the geodataframe on a map using the scatter_mapbox() method from plotly.express
fig = px.scatter_mapbox(gdf_stations,
                        lat=gdf_stations.geometry.y,
                        lon=gdf_stations.geometry.x,
                        hover_name="id_velov",
                        zoom=12, mapbox_style="carto-positron")

## We remove margin around the map
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## Show the map
fig.show()

## 3. Data preprocessing

### 3.1. Adding information


Before we can analyze the history data, we want to add some information. For example, the initial dataset does not directly provide the trips (departures/arrivals) of the users but only the number of bikes or places available at a given time (per 5 minutes). To make an analysis of the frequentation or of the departure and arrival zones according to the time of the day or the week I calculated the departures and arrivals by 30min intervals.

From the 30 minutes intervals we can for example infer the daily number.




In [ ]:
## Make a copy of our dataframe, to be able to return to the initial data if necessary
df_sampled = df_bikes.copy()

#### 3.1.1 Calculation of the number of daily arrivals and departures


We can make calculations directly by grouping the rows with the `groupby()` method.

Which columns should be grouped together to be able to calculate the daily departures and arrivals?

In [ ]:
df_sampled.head()

In [ ]:
## The 'transform' method allows to apply a calculation to the original dataframe (not grouped). 
## In our case, we want to perform a sum on the columns departure30min and arrival30min.

## Complete the list of columns
df_sampled["daily_departure"] = df_sampled.groupby([****])['departure30min'].transform('sum')
df_sampled["daily_arrival"] = df_sampled.groupby([****])['arrival30min'].transform('sum')


In [ ]:
## Display 15 rows randomly to see the result
df_sampled.sample(15)

#### 3.1.2 Distinction between weekdays and weekends

In order to analyze the data we want to be able to distinguish weekdays from weekend days, for this we need to prepare the data in order to identify the weekend days.

1. We define a function that returns true when the date is a weekday and false when it is the weekend
2. We apply this function on each row of our dataframe

In [ ]:
## The function weekDay, takes 3 parameters: the year, the month and the day
def weekDay(year, month, day):
    ## This method returns true if the date is a day of the week, false otherwise
    ## Use the datetime() function and the weekday() method
    ## https://docs.python.org/fr/3/library/datetime.html#datetime.datetime
    
    ****
    

## Vectorize the function in order to apply it efficiently (in terms of calculation time) on the dataframe
isWeekDay = np.vectorize(weekDay)

In [ ]:
## Add a new column from the result of the function applied on all the rows of the dataframe
df_sampled['IsWeekday'] = isWeekDay(****)


In [ ]:
## Display a sample of the dataframe
df_sampled.sample(10)

## 4. Saving the preprocessed dataset

In order to be able to reuse the dataset without redoing all the processing we save it in a CSV file.

Use the [to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) method from Pandas library to save the preprocessed dataframe into a CSV file. 

### 4.1 Method 1 : Export the dataframe as a csv

In [ ]:
df_sampled.to_csv('data-bikes-2.csv', index=False)

### 4.2 Method 2 : Export the dataframe as a csv in a zip file

In [ ]:
compression_opts = dict(method='zip', archive_name='data-bikes-2.csv')  
df_sampled.to_csv('data-bikes-2.zip', index=False, compression=compression_opts)